In [8]:
# =============== PHASE 0: SET UP =============== 

In [1]:
# PHASE 0.1 Import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os 
warnings.filterwarnings('ignore')

os.chdir('C:/Users/andno/OneDrive/Documents/MBAN 6110/Airbnb')

In [12]:
listings = pd.read_csv('listings.csv')
calendar = pd.read_csv('calendar.csv.gz')
reviews = pd.read_csv('reviews.csv')
neighbourhoods = pd.read_csv('neighbourhoods.csv')

In [ ]:
# Create an empty list to store successful chunks
chunks = []

# Try reading in chunks
chunk_size = 10000
try:
    for chunk in pd.read_csv('reviews.csv',
                             chunksize=chunk_size,
                             on_bad_lines='skip',
                             quoting=3,
                             encoding='utf-8',
                             engine='python'):
        chunks.append(chunk)
except Exception as e:
    print(f"Error during chunking: {e}")

# Combine all successfully read chunks
reviews = pd.concat(chunks, ignore_index=True)
print(f"Loaded {len(reviews)} rows successfully.")

In [ ]:
# Set indexes
listings.index = listings['id'].astype(str)
reviews.index = reviews['listing_id'].astype(str)

# Now you can join
merged = listings.join(reviews, how='inner', lsuffix='_list', rsuffix='_review')


In [ ]:
# =============== PHASE 1: DATA INSPECTION =============== 

In [4]:
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2539,Superfast Wi-Fi. Clean & quiet home by the park,2787,John,Brooklyn,Kensington,40.64529,-73.97238,Private room,NaN,30,9,2018-10-19,0.08,5,343,0,NaN
1,2595,Skylit Midtown Manhattan Spacious Studio,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,240.0,30,49,2022-06-21,0.26,3,365,0,NaN
2,5136,Spacious Family Friendly Duplex w/ Patio + Yard,7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,216.0,30,4,2023-08-20,0.03,1,37,0,NaN
3,6848,Only 2 stops to Manhattan studio,15991,Allen,Brooklyn,Williamsburg,40.70935,-73.95342,Entire home/apt,96.0,30,195,2025-04-01,1.00,1,182,4,NaN
4,6872,Uptown Sanctuary w/ Private Bath (Month to Month),16104,Kahshanna,Manhattan,East Harlem,40.80107,-73.94255,Private room,59.0,30,1,2022-06-05,0.03,2,83,0,NaN


In [5]:
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2539,2025-05-02,f,$299.00,NaN,30.0,730.0
1,2539,2025-05-03,f,$299.00,NaN,30.0,730.0
2,2539,2025-05-04,f,$299.00,NaN,30.0,730.0
3,2539,2025-05-05,f,$299.00,NaN,30.0,730.0
4,2539,2025-05-06,f,$299.00,NaN,30.0,730.0


In [11]:
neighbourhoods.head()

,neighbourhood_group,neighbourhood
0,Bronx,Allerton
1,Bronx,Baychester
2,Bronx,Belmont
3,Bronx,Bronxdale
4,Bronx,Castle Hill


In [13]:
reviews.head()

,listing_id,date
0,2539,2015-12-04
1,2539,2016-08-27
2,2539,2016-10-01
3,2539,2017-02-20
4,2539,2017-03-19


In [14]:
listings.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

In [ ]:
# =============== PHASE 2: DATA CLEANING =============== 

In [ ]:
merged['room_type'].count()

In [ ]:
listings['room_type'].count()

In [ ]:
merged = merged.dropna(subset=['price','neighbourhood_group','room_type'])
merged = merged.drop_duplicates()

In [ ]:
merged = merged[merged['price'] < merged['price'].quantile(0.99)]
merged = merged[merged['minimum_nights'] <= 365]
merged = merged[merged['date'] >= '2024-01-01']

In [ ]:
merged['date'] = pd.to_datetime(merged['date'], errors='coerce')

In [ ]:
print(merged.info())
print(merged[['date', 'price','minimum_nights']].describe())

In [ ]:
# =============== PHASE 3: DATA VISUALIZATION =============== 

In [ ]:
nyc_listings_neighbourhood = (
    merged.groupby('neighbourhood_group')['price']
    .mean()
    .sort_values()
    .reset_index()
)


In [ ]:
merged['date'] = pd.to_datetime(merged['date'], errors='coerce')
merged['month'] = merged['date'].dt.to_period('M')

monthly_price = (
    merged.groupby('month')['price']
    .mean()
    .reset_index()
)

monthly_price['month'] = monthly_price['month'].dt.to_timestamp()
monthly_price = monthly_price[monthly_price['month'] < '2025-05-01']


In [ ]:
# Step 1: Define buckets
bins = [0, 2, 5, 10, 30, 10000]
labels = ['1–2', '3–5', '6–10', '11–30', '31+']
merged['min_night_bin'] = pd.cut(merged['minimum_nights'], bins=bins, labels=labels)

# Step 2: Group and average by bin in most expensive borough
top_borough = (
    merged.groupby('neighbourhood_group')['price']
    .mean()
    .idxmax()
)

min_night_summary = (
    merged[merged['neighbourhood_group'] == top_borough]
    .groupby('min_night_bin')['price']
    .mean()
    .reset_index()
)



In [ ]:
# Extract the month for grouping
merged['month'] = merged['date'].dt.to_period('M')
# Group by month and borough to calculate average price
monthly_prices = (
    merged.groupby(['month', 'neighbourhood_group'])['price']
    .mean()
    .reset_index()
)

In [ ]:
# Convert 'month' to datetime for plotting
monthly_prices['month'] = monthly_prices['month'].dt.to_timestamp()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_prices, x='month', y='price', hue='neighbourhood_group')
plt.title('Monthly Average Price Trend by Borough')
plt.xlabel('Month')
plt.ylabel('Average Price ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
sns.barplot(data=nyc_listings_neighbourhood, x='price', y='neighbourhood_group')
plt.title('Average Airbnb Price by NYC Borough')
plt.xlabel('Average Price ($)')
plt.ylabel('Borough')
plt.show()


In [ ]:
borough_room_avg = (
    merged.groupby(['neighbourhood_group', 'room_type'])['price']
    .mean()
    .reset_index()
)

plt.figure(figsize=(12, 6))
sns.barplot(data=borough_room_avg, x='neighbourhood_group', y='price', hue='room_type')
plt.title('Avg Airbnb Price by Borough and Room Type')
plt.ylabel('Avg Price ($)')
plt.xlabel('Borough')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
borough_room_count = (
    merged.groupby(['neighbourhood_group', 'room_type'])
    .size()
    .reset_index(name='listing_count')
)

plt.figure(figsize=(12, 6))
sns.barplot(data=borough_room_count, x='neighbourhood_group', y='listing_count', hue='room_type')
plt.title('Listing Volume by Room Type & Borough')
plt.ylabel('Listing Count')
plt.xlabel('Borough')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:


# Line chart for avg price
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_price, x='month', y='price')
plt.title('Average Airbnb Price Over Time (Monthly)')
plt.xlabel('Month')
plt.ylabel('Average Price ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
monthly_reviews = merged.groupby('month')['number_of_reviews'].sum().reset_index()
monthly_reviews = monthly_reviews[monthly_reviews['month'] < '2025-05-01']

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(monthly_reviews['month'].astype(str), monthly_reviews['number_of_reviews'])
plt.xticks(rotation=45)
plt.title('Monthly Total Review Activity (Demand Proxy)')
plt.xlabel('Month')
plt.ylabel('Total Reviews')
plt.tight_layout()
plt.show()

In [ ]:
listings_per_month = merged.groupby('month')['id'].nunique().reset_index()
listings_per_month = listings_per_month[listings_per_month['month'] < '2025-05-04']

plt.figure(figsize=(12, 6))
plt.plot(listings_per_month['month'].astype(str), listings_per_month['id'])
plt.title('Monthly Unique Listings in NYC (Supply)')
plt.xlabel('Month')
plt.ylabel('Unique Listing Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
monthly_reviews = merged.groupby('month')['number_of_reviews'].sum().reset_index()
monthly_prices = merged.groupby('month')['price'].mean().reset_index()

# Merge both
trend_df = pd.merge(monthly_reviews, monthly_prices, on='month')

fig, ax1 = plt.subplots(figsize=(12, 6))

ax2 = ax1.twinx()
ax1.plot(trend_df['month'].astype(str), trend_df['number_of_reviews'], color='blue', label='Total Reviews')
ax2.plot(trend_df['month'].astype(str), trend_df['price'], color='red', label='Avg Price')

ax1.set_xlabel('Month')
ax1.set_ylabel('Total Reviews', color='blue')
ax2.set_ylabel('Average Price ($)', color='red')
plt.title('Demand vs. Price Over Time')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

